# K Nearest Neighbor 

STEP 1: Cgoose the number K of neighbors

STEP 2: Take the K nearest neighbors of the new data point, according to your distance metric

STEP 3: Among these K neighbors, count the number of data points to each category

STEP 4: Assign the new data point to the category where you counted the most neighbors

### 2. Importing and preperation of data

In [1]:
import numpy as np
import pandas as pd

NOTE: Iris dataset includes three iris species with 50 samples each as well as some properties about each flower. One flower species is linearly separable from the other two, but the other two are not linearly separable from each other.

In [2]:
# Importing the dataset
dataset = pd.read_csv('../../00_Datasets/Iris.csv')

In [3]:
dataset.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [21]:
dataset.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


NOTE: As we can see dataset contain six columns: Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm and Species. The actual features are described by columns 1-4. Last column contains labels of samples. Firstly we need to split data into two arrays: X (features) and y (labels).

In [16]:
feature_columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm']
X = dataset[feature_columns].values
y = dataset['Species'].values

# Alternative way of selecting features and labels arrays:
# X = dataset.iloc[:, 1:5].values
# y = dataset.iloc[:, 5].values

NOTE: As we can see labels are categorical. KNeighborsClassifier does not accept string labels. We need to use LabelEncoder to transform them into numbers. Iris-setosa correspond to 0, Iris-versicolor correspond to 1 and Iris-virginica correspond to 2.

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

Let's split dataset into training set and test set, to check later on whether or not our classifier works correctly.

In [27]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [29]:
X_test

array([[ 5.8,  2.8,  5.1,  2.4],
       [ 6. ,  2.2,  4. ,  1. ],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 7.3,  2.9,  6.3,  1.8],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 6.3,  3.3,  6. ,  2.5],
       [ 5. ,  3.5,  1.3,  0.3],
       [ 6.7,  3.1,  4.7,  1.5],
       [ 6.8,  2.8,  4.8,  1.4],
       [ 6.1,  2.8,  4. ,  1.3],
       [ 6.1,  2.6,  5.6,  1.4],
       [ 6.4,  3.2,  4.5,  1.5],
       [ 6.1,  2.8,  4.7,  1.2],
       [ 6.5,  2.8,  4.6,  1.5],
       [ 6.1,  2.9,  4.7,  1.4],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 6. ,  2.9,  4.5,  1.5],
       [ 5.5,  2.6,  4.4,  1.2],
       [ 4.8,  3. ,  1.4,  0.3],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.6,  2.8,  4.9,  2. ],
       [ 5.6,  3. ,  4.5,  1.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 6.2,  2.8,  4.8,  1.8],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.8,  1.9,  0.4],
       [ 6.2,  2.9,  4.3,  1.3],
       [ 5. ,  2.3,  3.3,  1. ],
       [ 5. ,  3.4,  1.6,  0.4]])

### 3. Bibliography

1. MIT Lecture: https://www.youtube.com/watch?v=09mb78oiPkA
2. Iris dataset: https://www.kaggle.com/uciml/iris